<a href="https://colab.research.google.com/github/sreent/large-language-model/blob/main/1%20LLMs%20with%20Hugging%20Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>